In [1]:
# load all packages in Julia
import Pkg; Pkg.add("VegaLite"); Pkg.add("PrettyTables")
using JuMP, HiGHS
using Plots;
using VegaLite  # to make some nice plots
using DataFrames, CSV, PrettyTables
ENV["COLUMNS"]=120; # Set so all columns of DataFrames and Matrices are displayed

    Updating registry at `C:\Users\44780\.julia\registries\General.toml`
   Resolving package versions...
  No Changes to `C:\Users\44780\.julia\environments\v1.9\Project.toml`
  No Changes to `C:\Users\44780\.julia\environments\v1.9\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\44780\.julia\environments\v1.9\Project.toml`
  No Changes to `C:\Users\44780\.julia\environments\v1.9\Manifest.toml`


In [48]:
#open csv files to define sets, lines, gens and and loads/demand
datadir = joinpath("ReducedNetwork") 
gens = CSV.read(joinpath(datadir,"UpdateGenRedNet.csv"), DataFrame);
lines = CSV.read(joinpath(datadir,"NoParTransRedNet.csv"), DataFrame);
loads = CSV.read(joinpath(datadir,"demandRedNet.csv"), DataFrame);


# Rename all columns to lowercase (by convention)
for f in [gens, lines, loads]
    rename!(f,lowercase.(names(f)))
end

# create generator ids #changed to read columns rather than rows
gens.id = 1:nrow(gens);

# create line ids 
lines.id = 1:nrow(lines);
# add set of rows for reverse direction with same parameters

lines2 = copy(lines)
lines2.f = lines2.fromnode
lines2.fromnode = lines.tonode
lines2.tonode = lines.fromnode
lines2 = lines2[:,names(lines)]
append!(lines,lines2)

lines.capacity = 10*lines.capacity
# calculate simple susceptance, ignoring resistance as earlier 
#lines.b = 1 ./ lines.reactance

# keep only a single time period
#loads = loads[:,["connnode","interval-1_load"]]
#rename!(loads,"interval-1_load" => "demand");

lines

Row,fromnode,tonode,resistance,reactance,contingencymarked,capacity,id,b
,Int64,Int64,Float64,Float64,Int64,Int64,Int64,Float64
1,1,2,0.0122,0.02,1,5250,1,0.0856
2,1,3,0.007,0.15,1,1320,2,0.052
3,2,4,0.0004,0.065,1,7600,3,0.4454
4,3,4,0.003,0.041,1,6480,4,0.0044
5,4,6,0.0013,0.023,1,15000,5,0.1496
6,4,7,0.001,0.024,1,10000,6,0.125
7,4,5,0.0021,0.0135,1,10900,7,0.1538
8,7,6,0.00151,0.01613,1,13900,8,0.59296
9,7,8,1.0e-6,0.0001,1,22000,9,1.0e-6


In [31]:
sum(gens.pgmax)

237270.089959114

In [32]:
sum(loads.demand)

-73847.1343515

In [5]:
loads

Row,connnode,demand,column3,column4,column5,column6,column7
,Int64,Float64,Missing,Missing,Missing,String31,Int64
1,1,-390.3,missing,missing,missing,Beauly,1
2,2,-624.695,missing,missing,missing,Peterhead,2
3,3,-202.172,missing,missing,missing,Errochty,3
4,4,-1828.53,missing,missing,missing,Denny/Bonnybridge,4
5,7,-2168.74,missing,missing,missing,Neilston,7
6,6,-660.162,missing,missing,missing,Strathaven,6
7,5,-572.218,missing,missing,missing,Torness,5
8,10,-162.219,missing,missing,missing,Eccles,10
9,9,-581.174,missing,missing,missing,Harker,9


In [34]:
#attempt 2 using PyPSA-GB network - works properly for the 36 bus network
#=
Function to solve DC OPF problem using IEEE test cases
Inputs:
    gen_info -- dataframe with generator info
    line_info -- dataframe with transmission lines info
    loads  -- dataframe with load info
=#
function dcopf_ieee(gens, lines, loads)
    DCOPF = Model(HiGHS.Optimizer) # You could use Clp as well, with Clp.Optimizer
    
    # Define sets based on data
      # Set of generator buses
    G = gens.id #the way it defines gens are by looking at which bus it's connected to - list gens as unique elements then create another matrix showing which gens are associated with which node in the system
                #set to id instead of connnode
    
      # Set of all nodes
    N = sort(union(unique(lines.fromnode), 
            unique(lines.tonode))) 
    
      # sets J_i and G_i will be described using dataframe indexing below

    # Define per unit base units for the system 
    # used to convert from per unit values to standard unit
    # values (e.g. p.u. power flows to MW/MVA)
    baseMVA = 100 # base MVA is 100 MVA for this system 
    
    # Decision variables   
    @variables(DCOPF, begin
        GEN[G]  >= 0     # generation        
        # Note: we assume Pmin = 0 for all resources for simplicty here
        THETA[N]         # voltage phase angle of bus
        FLOW[N,N]        # flows between all pairs of nodes
    end)
    
    # Create slack bus with reference angle = 0; use bus 1 with generator
    fix(THETA[1],0)
                
    # Objective function
    @objective(DCOPF, Min, 
        sum( gens[g,:c1] * GEN[g] for g in G)
    )
    
 # Supply demand balances
    @constraint(DCOPF, cBalance[i in N], 
        sum(GEN[g] for g in gens[gens.connnode .== i,:id]) 
            + sum(load for load in loads[loads.connnode .== i,:demand]) 
        == sum(FLOW[i,j] for j in lines[lines.fromnode .== i,:tonode])
    )

    # Max generation constraint
    @constraint(DCOPF, cMaxGen[g in G],
                    GEN[g] <= gens[g,:pgmax])
    
    # Flow constraints on each branch 
    @constraint(DCOPF, cLineFlows[l in 1:nrow(lines)],
            FLOW[lines[l,:fromnode],lines[l,:tonode]] == 
            baseMVA * lines[l,:b] * 
            (THETA[lines[l,:fromnode]] - THETA[lines[l,:tonode]])
    )
    
    # Max line flow constraints
    @constraint(DCOPF, cLineLimits[l in 1:nrow(lines)], 
            FLOW[lines[l,:fromnode],lines[l,:tonode]] <=
            lines[l,:capacity]
    ) 


    # Solve statement (! indicates runs in place)
    optimize!(DCOPF)

    # Output variables
    generation = DataFrame(
        node = gens.connnode,
        gen = value.(GEN).data[gens.connnode]
        )
    
    angles = value.(THETA).data
    
    flows = DataFrame(
        fbus = lines.fromnode,
        tbus = lines.tonode,
        flow = baseMVA * lines.b .* (angles[lines.fromnode] .- 
                        angles[lines.tonode]))
    
    # We output the marginal values of the demand constraints, 
    # which will in fact be the prices to deliver power at a given bus.
    prices = DataFrame(
        node = N,
        value = dual.(cBalance).data)

    # Return the solution and objective as named tuple
    return (
        generation = generation, 
        angles,
        flows,
        prices,
        cost = objective_value(DCOPF),
        status = termination_status(DCOPF)
    )
end

dcopf_ieee (generic function with 1 method)

In [20]:
sum(loads.demand)

-56325.86

In [49]:
solution = dcopf_ieee(gens, lines, loads);

Running HiGHS 1.5.3 [date: 1970-01-01, git hash: 45a127b78]
Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
131 rows, 456 cols, 730 nonzeros
126 rows, 255 cols, 523 nonzeros
126 rows, 255 cols, 523 nonzeros
Presolve : Reductions: rows 126(-442); columns 255(-950); elements 523(-657)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -7.1988072714e+00 Pr: 126(5.72008e+06); Du: 0(2.68791e-11) 0s
        134     7.3823499351e+04 Pr: 0(0); Du: 0(2.91767e-13) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 134
Objective value     :  7.3823499351e+04
HiGHS run time      :          0.01


In [50]:
solution.generation #pypsa-gb results below

Row,node,gen
,Int64,Float64
1,1,0.0
2,1,0.0
3,1,0.0
4,1,0.0
5,1,0.0
6,1,0.0
7,1,0.0
8,1,0.0
9,1,0.0


In [52]:
gentest = solution.generation

Row,node,gen
,Int64,Float64
1,1,0.0
2,1,0.0
3,1,0.0
4,1,0.0
5,1,0.0
6,1,0.0
7,1,0.0
8,1,0.0
9,1,0.0


In [51]:
using CSV

In [54]:
CSV.write(joinpath(datadir,"GenTestCap4.csv"), gentest)

"ReducedNetwork\\GenTestCap4.csv"

In [11]:
sum(solution.generation['gen'])

LoadError: syntax: character literal contains multiple characters

In [41]:
solution.prices

Row,node,value
,Int64,Float64
1,1,1.0
2,2,1.0
3,3,1.0
4,4,1.0
5,5,1.0
6,6,1.0
7,7,1.0
8,8,1.0
9,9,1.0


In [42]:
solution.flows

Row,fbus,tbus,flow
,Int64,Int64,Float64
1,1,2,-540.473
2,1,3,150.173
3,2,4,-1165.17
4,3,4,-51.9987
5,4,6,-1423.11
6,4,7,-604.482
7,4,5,-1018.1
8,7,6,-2773.23
9,7,8,0.00705203
